In [1]:
!pip install dgl==1.0.1+cu117 -f https://data.dgl.ai/wheels/cu117/repo.html

Looking in links: https://data.dgl.ai/wheels/cu117/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: dgl
    Found existing installation: dgl 1.1.2
    Uninstalling dgl-1.1.2:
      Successfully uninstalled dgl-1.1.2


In [2]:
import os, torch, time
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

torch.cuda.is_available()

True

In [3]:
gcn_msg = fn.copy_u(u='h', out='msg')
gcn_reduce = fn.sum(msg='msg', out = 'h')
class GCNlayer(nn.Module):
  def __init__(self, in_feats, out_feats):
    super().__init__()
    self.linear = nn.Linear(in_feats, out_feats)

  def forward(self, g, features):
    with g.local_scope():
      g.ndata['h'] = features
      g.update_all(gcn_msg, gcn_reduce)
      h = g.ndata['h']
      return self.linear(h)

In [5]:
from dgl.data import CoraGraphDataset
dataset = CoraGraphDataset()
g = dataset[0].to('cuda')
num_nodes, num_feats = g.ndata['feat'].shape
num_classes = len(torch.unique(g.ndata['label']))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [6]:
class GCN(nn.Module):
  def __init__(self, num_feats, hidden_dim, num_classes):
    super().__init__()
    self.gcn1 = GCNlayer(num_feats, hidden_dim)
    self.gcn2 = GCNlayer(hidden_dim, num_classes)

  def forward(self, g, x):
    x = F.relu(self.gcn1(g, x))
    x = self.gcn2(g, x)
    return x

model = GCN(num_feats=num_feats, hidden_dim=16, num_classes=num_classes).to('cuda')
print(model)

GCN(
  (gcn1): GCNlayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (gcn2): GCNlayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [7]:
def train(model, num_epochs, optimizer):
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]

    g.add_edges(g.nodes(), g.nodes()) # Add edges between each node and itself to preserve old node representations

    best_train_acc = best_val_acc = 0
    for e in range(num_epochs):
      t0 = time.time()
      logits = model(g, features)
      preds = logits.argmax(1)
      loss = F.cross_entropy(logits[train_mask], labels[train_mask])
      train_preds, val_preds = preds[train_mask], preds[val_mask]
      train_acc, val_acc = (preds[train_mask] == labels[train_mask]).float().mean(), (preds[val_mask] == labels[val_mask]).float().mean()
      best_train_acc = train_acc if train_acc > best_train_acc else best_train_acc
      best_val_acc = val_acc if val_acc > best_val_acc else best_val_acc

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      t_p_e = time.time() - t0

      if e % 5 == 0:
        print(f'Finished training of epoch {e}/{num_epochs} in {t_p_e:.3f}s,\n \
        CE loss is {loss:3f}, training accuracy is {train_acc:.3f}, validation accuracy is {val_acc:.3f}\n \
        best train accuracy is {best_train_acc:.3f}, best validation accuracy is {best_val_acc:.3f}.')

model = GCN(num_feats=num_feats, hidden_dim=16, num_classes=num_classes).to('cuda')
num_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train(model=model, num_epochs=num_epochs, optimizer=optimizer)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Finished training of epoch 0/100 in 4.228s,
         CE loss is 1.955010, training accuracy is 0.093, validation accuracy is 0.066
         best train accuracy is 0.093, best validation accuracy is 0.066.
Finished training of epoch 5/100 in 0.004s,
         CE loss is 1.831289, training accuracy is 0.221, validation accuracy is 0.126
         best train accuracy is 0.221, best validation accuracy is 0.126.
Finished training of epoch 10/100 in 0.004s,
         CE loss is 1.709802, training accuracy is 0.321, validation accuracy is 0.176
         best train accuracy is 0.321, best validation accuracy is 0.176.
Finished training of epoch 15/100 in 0.004s,
         CE loss is 1.604005, training accuracy is 0.493, validation accuracy is 0.298
         best train accuracy is 0.493, best validation accuracy is 0.298.
Finished training of epoch 20/100 in 0.004s,
         CE loss is 1.511066, training accuracy is 0.650, validation accuracy is 0.416
         best train accuracy is 0.650, best va